<a href="https://colab.research.google.com/github/ellieb0826/myuniverse/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/datasets

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/datasets


In [75]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv(r't_kjv.csv')

In [76]:
df.rename(columns = {'b':'book', 'c':'chapter', 'v':'verse', 't':'content'}, inplace=True)

In [77]:
key = pd.read_csv(r'key_english.csv')

book_key = {b: n for b, n in zip(key['b'], key['n'])}
df['book'] = df['book'].map(book_key)

In [78]:
print(df)

             id        book  chapter  verse  \
0       1001001     Genesis        1      1   
1       1001002     Genesis        1      2   
2       1001003     Genesis        1      3   
3       1001004     Genesis        1      4   
4       1001005     Genesis        1      5   
...         ...         ...      ...    ...   
31098  66022017  Revelation       22     17   
31099  66022018  Revelation       22     18   
31100  66022019  Revelation       22     19   
31101  66022020  Revelation       22     20   
31102  66022021  Revelation       22     21   

                                                 content  
0      In the beginning God created the heaven and th...  
1      And the earth was without form, and void; and ...  
2      And God said, Let there be light: and there wa...  
3      And God saw the light, that it was good: and G...  
4      And God called the light Day, and the darkness...  
...                                                  ...  
31098  And the Spirit 

In [79]:
# vectorize text
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(list(df['content']), show_progress_bar=True)

Batches:   0%|          | 0/972 [00:00<?, ?it/s]

In [80]:
cosine_data = pd.DataFrame(cosine_similarity(embeddings))

In [81]:
def recommend_verses(df, input_book, input_ch, input_v):
    input = df[(df['book'] == input_book) & (df['chapter'] == input_ch) & (df['verse']==input_v)]
    input_idx = input.index[0]
    input = df.iloc[input_idx]
    input_content = input['content']
    print(input_content)


    recomm_idx = cosine_data.iloc[input_idx].sort_values(ascending=False).index.tolist()[1:4]
    recomm_verses = df['content'].iloc[recomm_idx].values

    for i, idx in enumerate(recomm_idx):
        book = df['book'].iloc[idx]
        chapter = df['chapter'].iloc[idx]
        verse = df['verse'].iloc[idx]
        print('The #', i+1 , 'recommendation is', book, chapter, verse)
        print(recomm_verses[i])

    return recomm_idx, recomm_verses  
  
    

In [82]:
recommend_verses(df, 'Proverbs', 3, 5)

Trust in the LORD with all thine heart; and lean not unto thine own understanding.
The # 1 recommendation is Psalms 112 7
He shall not be afraid of evil tidings: his heart is fixed, trusting in the LORD.
The # 2 recommendation is Proverbs 2 2
So that thou incline thine ear unto wisdom, and apply thine heart to understanding;
The # 3 recommendation is Proverbs 23 17
Let not thine heart envy sinners: but be thou in the fear of the LORD all the day long.


([15810, 16435, 17061],
 array(['He shall not be afraid of evil tidings: his heart is fixed, trusting in the LORD.',
        'So that thou incline thine ear unto wisdom, and apply thine heart to understanding;',
        'Let not thine heart envy sinners: but be thou in the fear of the LORD all the day long.'],
       dtype=object))